# Imports and Defaults

In [1]:
import json
import os
import zipfile

import bridgestan as bs
from cmdstanpy import CmdStanModel
import numpy as np

In [6]:
seed = 1234
rng = np.random.default_rng(seed)

num_chains = 100
num_samples = 100_0000
num_dimensions = 50

model_name = f"gaussian_mixture"
model_path = model_name + ".stan"
data_path = model_name + ".json"

In [7]:
mu1 = np.zeros(num_dimensions)
mu2 = np.zeros(num_dimensions)
mu2[0] = 8

x1 = rng.normal(mu1, 1, size=(num_samples, num_dimensions))
x2 = rng.normal(mu2, 1, size=(num_samples, num_dimensions))
idx = rng.choice([0, 1], num_samples, p=[0.3, 0.7])
y = np.where(idx[:, None] == 0, x1, x2)

In [8]:
# extend the above code to multiple chains
mu1 = np.zeros(num_dimensions)
mu2 = np.zeros(num_dimensions)
mu2[0] = 8

x1 = rng.normal(mu1, 1, size=(num_chains, num_samples, num_dimensions))
x2 = rng.normal(mu2, 1, size=(num_chains, num_samples, num_dimensions))
idx = rng.choice([0, 1], (num_chains, num_samples), p=[0.3, 0.7])
y = np.where(idx[:, :, None] == 0, x1, x2) # [num_chains, num_samples, num_dimensions]

In [9]:
y_mean, y_std = y.mean(axis=(0, 1)), y.std(axis=(0, 1))
print(y_mean, y_std)

[ 5.59997864e+00 -4.57118206e-05  7.36651941e-05  4.82711098e-05
  2.33247320e-04  7.34790654e-05  1.55317763e-06  1.13004184e-04
 -4.49753728e-05 -1.00919667e-04  1.58804272e-05  2.91430143e-05
 -1.99471577e-04 -2.32670000e-04 -9.26425109e-05  2.53697471e-05
  1.41952642e-04  1.79068355e-04 -1.05318504e-05  5.31741555e-06
 -3.19313183e-05 -6.05381337e-05  3.74114203e-05  1.20431138e-04
 -9.44835689e-05  7.16442759e-05 -1.54513266e-04 -5.19413818e-05
 -3.50432414e-06 -8.11711165e-05 -9.63501799e-05  2.74520969e-05
 -3.04331962e-05 -6.64586552e-05  1.97792150e-04 -1.07098567e-04
 -1.51104250e-05 -9.44270330e-05 -6.51835156e-05 -7.74844867e-05
  1.22236923e-04 -2.81569096e-04 -5.22731551e-05  1.81172776e-04
  6.19922136e-05 -1.22079783e-04  3.68960499e-05  1.37988568e-04
 -6.20498563e-05 -3.40600046e-05] [3.80007463 1.00008617 0.99988475 1.00000747 0.99991336 0.99999733
 0.99996642 0.99991124 1.00008255 0.99988525 0.99998379 0.99994979
 1.00004612 1.00002428 0.99993017 0.99990086 1.00005

# Generate Reference Draws

In [4]:
v = rng.normal(0, 3, size=(num_chains, num_samples, 1))
theta = rng.normal(0, np.exp(v / 2), size=(num_chains, num_samples, num_dimensions - 1))
ref_draws_np = np.concatenate([v, theta], axis=2)

In [5]:
# convert from numpy 3d array to json format

param_names = ["v"] + [f"theta[{i}]" for i in range(1, num_dimensions)]

ref_draws = []
for chain in ref_draws_np:  # each chain is [n_samples, params_dim]
    chain_dict = {}
    for name, params in zip(param_names, chain.T):
        chain_dict[name] = params.tolist()
    ref_draws.append(chain_dict)

# save to file
with zipfile.ZipFile(f"{model_name}.ref_draws.json.zip", mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
    dumped_JSON: str = json.dumps(ref_draws, ensure_ascii=False, indent=4)
    zip_file.writestr(f"{model_name}.ref_draws.json", data=dumped_JSON)